<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d1f2e6773e1281ee4826b5faee8123bce9d174262b13d5887256ce8cabadfdd4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 14:14:04
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 37.30 K (0.26%)
Current PnL: -21.40 L (-14.03%)
CY Booked + Current PnL: -8.75 L (-5.74%)
-------------------
Total profit:  2.26 L
Total loss:  -23.67 L
-------------------
Total Booked + Current PnL: 18.23 L (14.51%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.38 L (61.73%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,64.0,X-LC,6.75,232986.0,22620.0,6151.0,1.05,10.75,2.64,13.68,37.0,3.68,1.64,29.05,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175530.0,34690.0,9795.0,0.70,24.63,5.58,31.59,66.0,3.54,1.24,57.64,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.01,86853.0,-13924.0,14018.0,-0.80,-13.82,16.14,0.09,245.0,-0.99,0.61,14.11,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.68,60.0,X-MC,6.28,150131.0,13998.0,16589.0,0.43,10.28,11.05,22.47,91.0,0.84,1.06,24.60,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.04,88500.0,-2178.0,17089.0,2.48,-2.40,19.31,16.44,101.0,-0.13,0.62,17.29,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.0,116.52,71.0,M-SC,5.85,141992.0,-32567.0,32601.0,14.04,-18.66,22.96,0.02,221.0,-1.00,1.00,23.51,OX40N,NTT,PAINTS
19,COFFEEDAY,80.0,-47.69,43.0,L-SC,27.36,77299.0,-36250.0,76302.0,3.26,-31.92,98.71,35.27,268.0,-0.48,0.54,89.19,XR,NTT,HOTELS
39,INFY,2275.0,-17.09,61.0,X-LC,4.86,323419.0,10531.0,161160.0,2.82,3.37,49.83,54.87,3.0,0.07,2.28,11.93,X40,BTT,IT
61,SAIL,228.0,57.98,71.0,M-MC,10.03,249328.0,24366.0,143289.0,2.69,10.83,57.47,74.53,192.0,0.17,1.75,46.58,XY24,BTT,STEEL
2,ABBOTINDIA,35195.0,-10.45,46.0,X-MC,3.04,88500.0,-2178.0,17089.0,2.48,-2.40,19.31,16.44,101.0,-0.13,0.62,17.29,X40,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,49.12,35.0,H-SC,14.41,136619.0,-42452.0,108175.0,-6.52,-23.71,79.18,36.71,135.0,-0.39,0.96,35.55,XR,NTT,BANKS
85,WHIRLPOOL,2270.0,-42.94,44.0,M-SC,6.98,118404.0,3606.0,90437.0,-3.96,3.14,76.38,81.92,223.0,0.04,0.83,40.17,XR,NTT,DURABLES
65,SHALBY,327.0,1054.70,36.0,M-SC,17.12,156958.0,-24185.0,69658.0,-3.83,-13.35,44.38,25.10,235.0,-0.35,1.10,25.79,XY24,NTT,HEALTHCARE
0,5PAISA,593.0,81.49,18.0,H-SC,15.80,122803.0,-95693.0,123884.0,-2.33,-43.80,100.88,12.90,153.0,-0.77,0.86,0.00,OX40N,NTT,FINANCE
55,RAJOOENG,143.1,-43.93,42.0,H-SC,16.69,85100.0,-17400.0,58004.0,-2.24,-16.98,68.16,39.61,114.0,-0.30,0.60,3.93,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.48,124123.0,-1174.0,60100.0,-2.12,-0.94,48.42,47.03,141.0,-0.02,0.87,55.41,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.01,86853.0,-13924.0,14018.0,-0.80,-13.82,16.14,0.09,245.0,-0.99,0.61,14.11,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.36,222147.0,-47520.0,83860.0,0.21,-17.62,37.75,13.47,138.0,-0.57,1.56,12.96,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,116.52,71.0,M-SC,5.85,141992.0,-32567.0,32601.0,14.04,-18.66,22.96,0.02,221.0,-1.00,1.00,23.51,OX40N,NTT,PAINTS
67,SIS,528.0,2040.93,50.0,H-SC,2.30,86407.0,-24625.0,48232.0,1.88,-22.18,55.82,21.26,156.0,-0.51,0.61,16.74,OX40N,NTT,MISC
17,CERA,9475.0,-25.54,38.0,H-SC,1.88,135850.0,-40053.0,82081.0,-1.51,-22.77,60.42,23.89,149.0,-0.49,0.96,17.32,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,69.0,M-LC,11.55,175530.0,34690.0,9795.0,0.7,24.63,5.58,31.59,66.0,3.54,1.24,57.64,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.98,71.0,M-MC,10.03,249328.0,24366.0,143289.0,2.69,10.83,57.47,74.53,192.0,0.17,1.75,46.58,XY24,BTT,STEEL
31,HINDZINC,730.22,29.80,54.0,M-LC,9.41,211127.0,6051.0,106514.0,2.29,2.95,50.45,54.89,52.0,0.06,1.49,27.76,X5K,ATH,METALS
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175530.0,34690.0,9795.0,0.70,24.63,5.58,31.59,66.0,3.54,1.24,57.64,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.94,44.0,M-SC,6.98,118404.0,3606.0,90437.0,-3.96,3.14,76.38,81.92,223.0,0.04,0.83,40.17,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,52.0,H-SC,5.25,129760.0,6424.0,125205.0,0.03,5.21,96.49,106.72,119.0,0.05,0.91,29.80,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,29.80,54.0,M-LC,9.41,211127.0,6051.0,106514.0,2.29,2.95,50.45,54.89,52.0,0.06,1.49,27.76,X5K,ATH,METALS
36,INDIAMART,4810.62,-53.52,52.0,H-SC,5.25,129760.0,6424.0,125205.0,0.03,5.21,96.49,106.72,119.0,0.05,0.91,29.80,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-42.94,44.0,M-SC,6.98,118404.0,3606.0,90437.0,-3.96,3.14,76.38,81.92,223.0,0.04,0.83,40.17,XR,NTT,DURABLES
86,WIPRO,420.00,-15.15,46.0,M-LC,5.59,148930.0,-2015.0,111474.0,1.57,-1.34,74.85,72.51,53.0,-0.02,1.05,4.61,XR,NTT,IT
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.48,124123.0,-1174.0,60100.0,-2.12,-0.94,48.42,47.03,141.0,-0.02,0.87,55.41,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
44,JSWINFRA,342.00,-25.18,24.0,X-MC,4.41,187988.0,-12359.0,42861.0,-0.61,-6.17,22.80,15.23,178.0,-0.29,1.32,24.09,X40N,NTT,REALTY
23,DMART,5391.45,-20.99,27.0,X-LC,7.17,201340.0,-9067.0,68234.0,0.39,-4.31,33.89,28.12,38.0,-0.13,1.42,18.30,X40N,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87142.0,-41528.0,86828.0,-1.13,-32.27,99.64,35.20,219.0,-0.48,0.61,0.02,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-14.27,31.0,X-LC,6.22,240940.0,-16123.0,51272.0,-0.19,-6.27,21.28,13.67,24.0,-0.31,1.70,13.74,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.34,36.0,X-MC,0.28,198585.0,-2235.0,70021.0,0.49,-1.11,35.26,33.76,80.0,-0.03,1.40,4.03,X40,ATH,FMCG
41,ITC,452.00,-38.77,44.0,X-LC,0.81,198387.0,-1751.0,22636.0,0.41,-0.87,11.41,10.44,4.0,-0.08,1.40,4.64,X40,NTT,FMCG
66,SIEMENS,4671.50,-4.32,44.0,H-LC,1.54,153930.0,-32165.0,79643.0,1.49,-17.28,51.74,25.51,15.0,-0.40,1.08,13.43,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.30,39.0,X-MC,3.28,305970.0,-22021.0,128569.0,0.59,-6.71,42.02,32.48,92.0,-0.17,2.15,3.94,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.52,58.0,X-LC,3.81,242434.0,8290.0,40341.0,-1.37,3.54,16.64,20.77,86.0,0.21,1.71,10.70,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87142.0,-41528.0,86828.0,-1.13,-32.27,99.64,35.20,219.0,-0.48,0.61,0.02,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49058.0,-15948.0,166876.0,-1.12,-24.53,340.16,232.18,198.0,-0.10,0.35,0.40,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,38.0,X-MC,7.53,80300.0,-3210.0,23239.0,1.03,-3.84,28.94,23.98,82.0,-0.14,0.57,1.75,X40,ATH,APPARELS
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239880.0,-21739.0,140953.0,-0.38,-8.31,58.76,45.57,7.0,-0.15,1.69,2.07,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.18,217550.0,-45815.0,155135.0,0.38,-17.40,71.31,41.51,84.0,-0.30,1.53,2.09,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49058.0,-15948.0,166876.0,-1.12,-24.53,340.16,232.18,198.0,-0.10,0.35,0.40,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,38.0,X-MC,7.53,80300.0,-3210.0,23239.0,1.03,-3.84,28.94,23.98,82.0,-0.14,0.57,1.75,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.85,40.0,X-SC,4.20,80661.0,-64499.0,140423.0,-0.56,-44.43,174.09,52.31,136.0,-0.46,0.57,7.97,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87142.0,-41528.0,86828.0,-1.13,-32.27,99.64,35.20,219.0,-0.48,0.61,0.02,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.04,88500.0,-2178.0,17089.0,2.48,-2.40,19.31,16.44,101.0,-0.13,0.62,17.29,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.55,284641.0,-61315.0,128003.0,1.21,-17.72,44.97,19.28,1.0,-0.48,2.00,5.23,X40,ATH,IT
39,INFY,2275.00,-17.09,61.0,X-LC,4.86,323419.0,10531.0,161160.0,2.82,3.37,49.83,54.87,3.0,0.07,2.28,11.93,X40,BTT,IT
41,ITC,452.00,-38.77,44.0,X-LC,0.81,198387.0,-1751.0,22636.0,0.41,-0.87,11.41,10.44,4.0,-0.08,1.40,4.64,X40,NTT,FMCG
83,VBL,671.64,-17.48,50.0,X-LC,5.65,297340.0,-18502.0,131157.0,-0.88,-5.86,44.11,35.67,5.0,-0.14,2.09,7.18,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239880.0,-21739.0,140953.0,-0.38,-8.31,58.76,45.57,7.0,-0.15,1.69,2.07,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,27.36,77299.0,-36250.0,76302.0,3.26,-31.92,98.71,35.27,268.0,-0.48,0.54,89.19,XR,NTT,HOTELS
7,ASIANTILES,137.00,7400.00,64.0,L-SC,13.02,82315.0,-11495.0,88110.0,0.67,-12.25,107.04,81.67,269.0,-0.13,0.58,59.41,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,7.97,91830.0,-6150.0,27751.0,-0.41,-6.28,30.22,22.05,152.0,-0.22,0.65,33.48,XR,ATH,FINANCE
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.48,124123.0,-1174.0,60100.0,-2.12,-0.94,48.42,47.03,141.0,-0.02,0.87,55.41,MH,ATH,POWER
37,INDIGOPNTS,1408.00,116.52,71.0,M-SC,5.85,141992.0,-32567.0,32601.0,14.04,-18.66,22.96,0.02,221.0,-1.00,1.00,23.51,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.35,61.0,H-SC,4.82,148473.0,-34302.0,145370.0,-0.98,-18.77,97.91,60.77,125.0,-0.24,1.04,33.71,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,21.94,68.0,X-SC,6.44,215414.0,24408.0,33281.0,0.41,12.78,15.45,30.21,157.0,0.73,1.52,36.33,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7400.00,64.0,L-SC,13.02,82315.0,-11495.0,88110.0,0.67,-12.25,107.04,81.67,269.0,-0.13,0.58,59.41,XR,NTT,CERAMICS
6,ASIANPAINT,3465.66,-7.57,74.0,X-LC,5.61,233455.0,-18313.0,71531.0,1.50,-7.27,30.64,21.13,27.0,-0.26,1.64,25.57,X40,ATH,PAINTS
68,SONACOMS,806.63,-31.01,69.0,M-SC,6.26,85838.0,-15321.0,55323.0,-0.12,-15.15,64.45,39.54,202.0,-0.28,0.60,21.04,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.82
1,25,44.57
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.63
LC    33.27
MC    23.11
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.98
X40      16.41
X40N     12.42
XY25     11.92
XR        9.75
AR        7.76
OX40N     6.59
X200      1.79
X5K       1.49
SR        1.03
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.04
X-LC    22.54
X-MC    18.72
M-SC    12.55
X-SC     5.59
M-LC     5.18
H-LC     4.56
H-MC     2.32
M-MC     1.75
L-SC     1.45
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-4.71,38.57
IT,12.57,-17.83,80.89
FINANCE,12.14,-9.56,59.22
MISC,7.77,-17.66,77.01
PAINTS,5.73,-13.35,30.25
ELECTRICAL,5.68,-11.86,51.52
BANKS,3.96,-20.14,90.03
INSURANCE,3.94,1.06,35.84
AUTO,2.71,-49.27,111.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3312945.0,22
XR,1271879.0,13
AR,1183593.0,9
X40,825243.0,12
X40N,704398.0,9
OX40N,549154.0,9
XY25,479141.0,8
SR,275090.0,2
X5K,106514.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3414283.0,24
M-SC,1260345.0,15
X-MC,1213788.0,14
X-LC,1078145.0,13
X-SC,555555.0,6
H-LC,290458.0,3
M-LC,273813.0,4
H-MC,258343.0,2
L-SC,254661.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1186602.0      6
           AR         839243.0      5
           XR         799051.0      7
M-SC       XY24       736522.0      6
X-MC       XY24       573959.0      4
X-LC       X40        443547.0      5
           XY24       312173.0      2
X-SC       X40N       301851.0      4
X-MC       X40        294868.0      6
H-SC       SR         275090.0      2
           OX40N      254197.0      3
X-LC       X40N       239732.0      3
H-LC       AR         220596.0      2
H-MC       XY24       183729.0      1
X-MC       XY25       182146.0      2
X-SC       XY24       166876.0      1
L-SC       XR         164412.0      2
X-MC       X40N       162815.0      2
M-MC       XY24       143289.0      1
M-SC       XR         139886.0      2
           OX40N      130094.0      4
           XY25       130089.0      1
           AR         123754.0      2
M-LC       XR         111474.0      1
           X5K        106514.0      1
L-SC       OX40N       90249.0      1
X-SC       X40         86828.0      1
X-LC       XY25        82693.0      3
H-MC       OX40N       74614.0      1
H-LC       X200        69862.0      1
H-SC       MH          60100.0      1
L-MC       XR          57056.0      1
M-LC       XY25        46030.0      1
L-LC       XY25        38183.0      1
M-LC       XY24         9795.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
